<a href="https://colab.research.google.com/github/kiiwee/opengen/blob/master/Phi-3/Phi_3_Medium_4K_Instruct_Inference_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="align-center">
  <a href="https://www.fontys.nl/"><img src="https://www.fontys.nl/static/design/FA845701-BD71-466E-9B3D-38580DFAD5B4-fsm/images/logo-inverted@2x.png" width="120"></a>
  <!-- <img src="https://i.imgur.com/zyfbV3r.png" width="20">
  <a href="https://www.arise-biodiversity.nl/"><img src="https://i.imgur.com/fwGepON.png" width="200"></a>
  <img src="https://i.imgur.com/zyfbV3r.png" width="20">
  <a href="https://diopsis.eu/"><img src="https://i.imgur.com/e6nutxp.png" width="180"></a> -->
</div>

# **In this notebook we will compare the results of the base Phi-3 Medium 4K Instruct model with the finetuned one on a website generation**

---

[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/kiiwee/opengen)

<a href="https://github.com/facebookresearch/detectron2"><img src="https://raw.githubusercontent.com/unslothai/unsloth/main/images/unsloth%20logo%20white%20text.png" width="80"></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

# ***First we test the original model:***

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # RoPE Scaling internally
dtype = None # Auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-medium-4k-instruct", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Mistral patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
alpaca_prompt = """You are a web developer who codes modern websites using HTML and Tailwind CSS. Use Tailwind CSS components. Reply only with the pure HTML/TailwindCSS code starting with <!DOCTYPE html> and ending with </html>

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [6]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "I want a landing page for a pet store with a navbar, a pricing section and a footer.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

You are a web developer who codes modern websites using HTML and Tailwind CSS. Use Tailwind CSS components. Reply only with the pure HTML/TailwindCSS code starting with <!DOCTYPE html> and ending with </html>

### Instruction:
I want a landing page for a pet store with a navbar, a pricing section and a footer.

### Input:


### Response:

```html

<!DOCTYPE html>

<html lang="en">

<head>

 <meta charset="UTF-8">

 <meta name="viewport" content="width=device-width, initial-scale=1.0">

 <title>Pet Store</title>

 <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">

</head>

<body class="bg-gray-100">

 <nav class="bg-white p-4 sticky top-0 relative shadow-md">

   <div class="container mx-auto">

     <div class="flex justify-between items-center">

       <div class="flex items-center space-x-4">

         <div class="text-2xl text-gray-700">Pet Store</div>

         <div class="hidden md:block">

           <div class="flex items-cente

In [10]:
from IPython.display import display, HTML
display(HTML("""<!DOCTYPE html>

<html lang="en">

<head>

 <meta charset="UTF-8">

 <meta name="viewport" content="width=device-width, initial-scale=1.0">

 <title>Pet Store</title>

 <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">

</head>

<body class="bg-gray-100">

 <nav class="bg-white p-4 sticky top-0 relative shadow-md">

   <div class="container mx-auto">

     <div class="flex justify-between items-center">

       <div class="flex items-center space-x-4">

         <div class="text-2xl text-gray-700">Pet Store</div>

         <div class="hidden md:block">

           <div class="flex items-center space-x-4">

             <a href="#" class="text-gray-600 hover:text-gray-900">Home</a>

             <a href="#" class="text-gray-600 hover:text-gray-900">Products</a>

             <a href="#" class="text-gray-60ia-600 hover:text-gray-900">About Us</a>

             <a href="#" class="text-gray-600 hover:text-gray-900">Contact</a>

           </div>

         </div>

       </div>

     </div>

   </div>

 </nav>


 <section class="py-10 max-w-4xl mx-auto px-4">

   <div class="text-center">

     <h1 class="text-4xl font-bold text-gray-900 mb-6">Pricing Plans</h1>

     <div class="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-4">

       <div class="bg-white rounded-lg shadow-md p-6">

         <h2 class="text-xl font-bold text-gray-900 mb-4">Basic Plan</h2>

         <p class="text-gray-600">$19.99/month</p>

         <ul class="mb-4">

           <li>Free Shipping</li>

           <li>10% Off on Products</li>

           <li>24/7 Customer Support</li>

         </ul>

       </div>

       <div class="bg-white rounded-lg shadow-md p-6">

         <h2 class="text-xl font-bold text-gray-900 mb-4">Premium Plan</h2>

         <p class="text-gray-600">$39.99/month</p>

         <ul class="mb-4">

           <li>Free Shipping</li>

           <li>20% Off on Products</li>

           <li>24/7 Customer Support</li>

           <li>Pet Health Insurance</li>

         </ul>

       </div>

     </div>

   </div>

 </section>


 <footer class="bg-gray-200 text-center p-4">

   <p>&copy; 2023 Pet Store. All rights reserved.</p>

 </footer>

</body>

</html>"""))

## And now with a longer maximum lenght:




In [4]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "I want a landing page for a pet store with a navbar, a pricing section and a footer.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

You are a web developer who codes modern websites using HTML and Tailwind CSS. Use Tailwind CSS components. Reply only with the pure HTML/TailwindCSS code starting with <!DOCTYPE html> and ending with </html>

### Instruction:
I want a landing page for a pet store with a navbar, a pricing section and a footer.

### Input:


### Response:

```html

<!DOCTYPE html>

<html lang="en">

<head>

 <meta charset="UTF-8">

 <meta name="viewport" content="width=device-width, initial-scale=1.0">

 <title>Pet Store</title>

 <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">

</head>

<body class="bg-gray-100">

 <nav class="bg-white p-4 sticky top-0 relative shadow-md">

   <div class="container mx-auto">

     <div class="flex justify-between items-center">

       <div class="flex items-center space-x-4">

         <div class="text-2xl text-gray-700">Pet Store</div>

         <div class="hidden md:block">

           <div class="flex items-cente

In [5]:
from IPython.display import display, HTML
display(HTML("""
<!DOCTYPE html>

<html lang="en">

<head>

 <meta charset="UTF-8">

 <meta name="viewport" content="width=device-width, initial-scale=1.0">

 <title>Pet Store</title>

 <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">

</head>

<body class="bg-gray-100">

 <nav class="bg-white p-4 sticky top-0 relative shadow-md">

   <div class="container mx-auto">

     <div class="flex justify-between items-center">

       <div class="flex items-center space-x-4">

         <div class="text-2xl text-gray-700">Pet Store</div>

         <div class="hidden md:block">

           <div class="flex items-center space-x-4">

             <a href="#" class="text-gray-600 hover:text-gray-900">Home</a>

             <a href="#" class="text-gray-600 hover:text-gray-900">Products</a>

             <a href="#" class="text-gray-60ia-600 hover:text-gray-900">About Us</a>

             <a href="#" class="text-gray-600 hover:text-gray-900">Contact</a>

           </div>

         </div>

       </div>

     </div>

   </div>

 </nav>


 <section class="py-10 max-w-4xl mx-auto px-4">

   <div class="text-center">

     <h1 class="text-4xl font-bold text-gray-900 mb-6">Pricing Plans</h1>

     <div class="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-4">

       <div class="bg-white rounded-lg shadow-md p-6">

         <h2 class="text-xl font-bold text-gray-900 mb-4">Basic Plan</h2>

         <p class="text-gray-600">$19.99/month</p>

         <ul class="mb-4">

           <li>Free Shipping</li>

           <li>10% Off on Products</li>

           <li>24/7 Customer Support</li>

         </ul>

       </div>

       <div class="bg-white rounded-lg shadow-md p-6">

         <h2 class="text-xl font-bold text-gray-900 mb-4">Premium Plan</h2>

         <p class="text-gray-600">$39.99/month</p>

         <ul class="mb-4">

           <li>Free Shipping</li>

           <li>20% Off on Products</li>

           <li>24/7 Customer Support</li>

           <li>Pet Health Insurance</li>

         </ul>

       </div>

     </div>

   </div>

 </section>


 <footer class="bg-gray-200 text-center p-4">

   <p>&copy; 2023 Pet Store. All rights reserved.</p>

 </footer>

</body>

</html>
"""))

## ⚠️ ***You need to restart the runtime session in order to clear-up VRAM space for the load of the second model.***

# ***And then the fine-tuned one:***

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # RoPE Scaling internally
dtype = None # Auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "kalinkov/phi3_lora_model",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Mistral patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.6 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [ ]:
alpaca_prompt = """You are a web developer who codes modern websites using HTML and Tailwind CSS. Use Tailwind CSS components. Reply only with the pure HTML/TailwindCSS code starting with <!DOCTYPE html> and ending with </html>

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [4]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "I need a landing page for a pet store with a navbar and pricing section.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

You are a web developer who codes modern websites using HTML and Tailwind CSS. Use Tailwind CSS components. Reply only with the pure HTML/TailwindCSS code starting with <!DOCTYPE html> and ending with </html>

### Instruction:
I need a landing page for a pet store with a navbar and pricing section.

### Input:


### Response:

```html<div>
	<section class="py-24 bg-blueGray-100">
		<div class="container px-4 mx-auto">
			<div class="flex flex-wrap -mx-4">
				<div class="w-full lg:w-1/2 px-4 mb-16 lg:mb-0">
					<div class="max-w-lg">
						<span class="text-sm text-blue-400 font-semibold uppercase">
							<span>Lorem ipsum</span>
							<span class="text-blue-400">Dolor sit amet consectutar</span>
						</span>
						<h2 class="mb-6 text-4xl md:text-5xl font-bold font-heading">
							Lorem ipsum dolor sit amet consectutar
						</h2>
						<p class="mb-8 text-lg leading-loose text-gray-500">
							Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh, pul

In [5]:
from IPython.display import display, HTML
display(HTML("""
<div>
	<section class="py-24 bg-blueGray-100">
		<div class="container px-4 mx-auto">
			<div class="flex flex-wrap -mx-4">
				<div class="w-full lg:w-1/2 px-4 mb-16 lg:mb-0">
					<div class="max-w-lg">
						<span class="text-sm text-blue-400 font-semibold uppercase">
							<span>Lorem ipsum</span>
							<span class="text-blue-400">Dolor sit amet consectutar</span>
						</span>
						<h2 class="mb-6 text-4xl md:text-5xl font-bold font-heading">
							Lorem ipsum dolor sit amet consectutar
						</h2>
						<p class="mb-8 text-lg leading-loose text-gray-500">
							Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh, pulvinar vitae aliquet nec,
							accumsan aliquet orci.
						</p>
						<div class="flex flex-wrap -mx-2">
							<div class="w-full md:w-1/2 px-2 mb-4 md:mb-0">
								<div class="flex items-center">
									<img
										class="h-12"
										src="https://shuffle.dev/uinel-assets/images/cta/avatar-woman.png"
										alt=""
									/>
									<div class="px-4">
										<h3 class="text-2xl font-bold font-heading">
											Lorem ipsum dolor sit amet consectutar
										</h3>
										<p class="text-lg leading-loose text-gray-500">
											Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh,
											pulvinar vitae aliquet nec, accumsan aliquet orci.
										</p>
									</div>
								</div>
							</div>
							<div class="w-full md:w-1/2 px-2 mb-4 md:mb-0">
								<div class="flex items-center">
									<img
										class="h-12"
										src="https://shuffle.dev/uinel-assets/images/cta/avatar-man.png"
										alt=""
									/>
									<div class="px-4">
										<h3 class="text-2xl font-bold font-heading">
											Lorem ipsum dolor sit amet consectutar
										</h3>
										<p class="text-lg leading-loose text-gray-500">
											Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh,
											pulvinar vitae aliquet nec, accumsan aliquet orci.
										</p>
									</div>
								</div>
							</div>
						</div>
					</div>
				</div>
				<div class="w-full lg:w-1/2 px-4 mb-16 lg:mb-0">
					<div class="max-w-lg">
						<span class="text-sm text-blue-400 font-semibold uppercase">
							<span>Lorem ipsum</span>
							<span class="text-blue-400">Dolor sit amet consectutar</span>
						</span>
						<h2 class="mb-6 text-4xl md:text-5xl font-bold font-heading">
							Lorem ipsum dolor sit amet consectutar
						</h2>
						<p class="mb-8 text-lg leading-loose text-gray-500">
							Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh, pulvinar vitae
							aliquet nec, accumsan aliquet orci.
						</p>
						<div class="flex flex-wrap -mx-2">
							<div class="w-full md:w-1/2 px-2 mb-4 md:mb-0">
								<div class="flex items-center">
									<img
										class="h-12"
										src="https://shuffle.dev/uinel-assets/images/cta/avatar-woman.png"
										alt=""
									/>
									<div class="px-4">
										<h3 class="text-2xl font-bold font-heading">
											Lorem ipsum dolor sit amet consectutar
										</h3>
										<p class="text-lg leading-loose text-gray-500">
											Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh,
											pulvinar vitae aliquet nec, accumsan aliquet orci.
										</p>
									</div>
								</div>
							</div>
							<div class="w-full md:w-1/2 px-2 mb-4 md:mb-0">
								<div class="flex items-center">
									<img
										class="h-12"
										src="https://shuffle.dev/uinel-assets/images/cta/avatar-man.png"
										alt=""
									/>
									<div class="px-4">
"""))

### The model has learned to not include the `<head>` segment, so now we can just add it before the generated code:
(`<script src="https://cdn.tailwindcss.com"></script>`)

In [6]:
from IPython.display import display, HTML
display(HTML("""<!DOCTYPE html>

<html lang="en">

<head>

 <meta charset="UTF-8">

 <meta name="viewport" content="width=device-width, initial-scale=1.0">

 <title>Pet Store</title>

 <script src="https://cdn.tailwindcss.com"></script>

</head>

<body class="bg-gray-100">
<div>
	<section class="py-24 bg-blueGray-100">
		<div class="container px-4 mx-auto">
			<div class="flex flex-wrap -mx-4">
				<div class="w-full lg:w-1/2 px-4 mb-16 lg:mb-0">
					<div class="max-w-lg">
						<span class="text-sm text-blue-400 font-semibold uppercase">
							<span>Lorem ipsum</span>
							<span class="text-blue-400">Dolor sit amet consectutar</span>
						</span>
						<h2 class="mb-6 text-4xl md:text-5xl font-bold font-heading">
							Lorem ipsum dolor sit amet consectutar
						</h2>
						<p class="mb-8 text-lg leading-loose text-gray-500">
							Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh, pulvinar vitae aliquet nec,
							accumsan aliquet orci.
						</p>
						<div class="flex flex-wrap -mx-2">
							<div class="w-full md:w-1/2 px-2 mb-4 md:mb-0">
								<div class="flex items-center">
									<img
										class="h-12"
										src="https://shuffle.dev/uinel-assets/images/cta/avatar-woman.png"
										alt=""
									/>
									<div class="px-4">
										<h3 class="text-2xl font-bold font-heading">
											Lorem ipsum dolor sit amet consectutar
										</h3>
										<p class="text-lg leading-loose text-gray-500">
											Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh,
											pulvinar vitae aliquet nec, accumsan aliquet orci.
										</p>
									</div>
								</div>
							</div>
							<div class="w-full md:w-1/2 px-2 mb-4 md:mb-0">
								<div class="flex items-center">
									<img
										class="h-12"
										src="https://shuffle.dev/uinel-assets/images/cta/avatar-man.png"
										alt=""
									/>
									<div class="px-4">
										<h3 class="text-2xl font-bold font-heading">
											Lorem ipsum dolor sit amet consectutar
										</h3>
										<p class="text-lg leading-loose text-gray-500">
											Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh,
											pulvinar vitae aliquet nec, accumsan aliquet orci.
										</p>
									</div>
								</div>
							</div>
						</div>
					</div>
				</div>
				<div class="w-full lg:w-1/2 px-4 mb-16 lg:mb-0">
					<div class="max-w-lg">
						<span class="text-sm text-blue-400 font-semibold uppercase">
							<span>Lorem ipsum</span>
							<span class="text-blue-400">Dolor sit amet consectutar</span>
						</span>
						<h2 class="mb-6 text-4xl md:text-5xl font-bold font-heading">
							Lorem ipsum dolor sit amet consectutar
						</h2>
						<p class="mb-8 text-lg leading-loose text-gray-500">
							Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh, pulvinar vitae
							aliquet nec, accumsan aliquet orci.
						</p>
						<div class="flex flex-wrap -mx-2">
							<div class="w-full md:w-1/2 px-2 mb-4 md:mb-0">
								<div class="flex items-center">
									<img
										class="h-12"
										src="https://shuffle.dev/uinel-assets/images/cta/avatar-woman.png"
										alt=""
									/>
									<div class="px-4">
										<h3 class="text-2xl font-bold font-heading">
											Lorem ipsum dolor sit amet consectutar
										</h3>
										<p class="text-lg leading-loose text-gray-500">
											Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh,
											pulvinar vitae aliquet nec, accumsan aliquet orci.
										</p>
									</div>
								</div>
							</div>
							<div class="w-full md:w-1/2 px-2 mb-4 md:mb-0">
								<div class="flex items-center">
									<img
										class="h-12"
										src="https://shuffle.dev/uinel-assets/images/cta/avatar-man.png"
										alt=""
									/>
									<div class="px-4">

  """))

## And now with the longer maximum lenght:

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "I want a landing page for a pet store with a navbar, a pricing section and a footer.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

You are a web developer who codes modern websites using HTML and Tailwind CSS. Use Tailwind CSS components. Reply only with the pure HTML/TailwindCSS code starting with <!DOCTYPE html> and ending with </html>

### Instruction:
I want a landing page for a pet store with a navbar, a pricing section and a footer.

### Input:


### Response:

```html<div>
	<section class="py-24 bg-blueGray-100">
		<div class="container px-4 mx-auto">
			<div class="flex flex-wrap -mx-4">
				<div class="w-full lg:w-1/2 px-4 mb-16 lg:mb-0">
					<div class="max-w-lg">
						<span class="text-sm text-blue-400 font-semibold uppercase">
							<span>Lorem ipsum</span>
							<span class="text-blue-400">Dolor sit amet consectutar</span>
						</span>
						<h2 class="mb-6 text-4xl md:text-5xl font-bold font-heading">
							Lorem ipsum dolor sit amet consectutar
						</h2>
						<p class="mb-8 text-lg leading-loose text-gray-500">
							Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque mas

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


In [7]:
from IPython.display import display, HTML
display(HTML("""<!DOCTYPE html>

<html lang="en">

<head>

 <meta charset="UTF-8">

 <meta name="viewport" content="width=device-width, initial-scale=1.0">

 <title>Pet Store</title>

 <script src="https://cdn.tailwindcss.com"></script>

</head>

<body class="bg-gray-100">
<div>
	<section class="py-24 bg-blueGray-100">
		<div class="container px-4 mx-auto">
			<div class="flex flex-wrap -mx-4">
				<div class="w-full lg:w-1/2 px-4 mb-16 lg:mb-0">
					<div class="max-w-lg">
						<span class="text-sm text-blue-400 font-semibold uppercase">
							<span>Lorem ipsum</span>
							<span class="text-blue-400">Dolor sit amet consectutar</span>
						</span>
						<h2 class="mb-6 text-4xl md:text-5xl font-bold font-heading">
							Lorem ipsum dolor sit amet consectutar
						</h2>
						<p class="mb-8 text-lg leading-loose text-gray-500">
							Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh, pulvinar vitae aliquet nec,
							accumsan aliquet orci.
						</p>
						<div class="flex flex-wrap -mx-2">
							<div class="w-full md:w-1/2 px-2 mb-4 md:mb-0">
								<div class="flex items-center">
									<img
										class="h-12"
										src="https://shuffle.dev/uinel-assets/images/cta/avatar-woman.png"
										alt=""
									/>
									<div class="ml-4">
										<h3 class="text-2xl font-bold font-heading">
											Lorem ipsum dolor sit amet consectutar
										</h3>
										<p class="text-lg leading-loose text-gray-500">
											Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh,
											pulvinar vitae aliquet nec, accumsan aliquet orci.
										</p>
									</div>
								</div>
							</div>
							<div class="w-full md:w-1/2 px-2">
								<div class="flex items-center">
									<img
										class="h-12"
										src="https://shuffle.dev/uinel-assets/images/cta/avatar-man.png"
										alt=""
									/>
									<div class="ml-4">
										<h3 class="text-2xl font-bold font-heading">
											Lorem ipsum dolor sit amet consectutar
										</h3>
										<p class="text-lg leading-loose text-gray-500">
											Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh,
											pulvinar vitae aliquet nec, accumsan aliquet orci.
										</p>
									</div>
								</div>
							</div>
						</div>
					</div>
				</div>
				<div class="w-full lg:w-1/2 px-4 mb-16 lg:mb-0">
					<div class="max-w-lg">
						<span class="text-sm text-blue-400 font-semibold uppercase">
							<span>Lorem ipsum</span>
							<span class="text-blue-400">Dolor sit amet consectutar</span>
						</span>
						<h2 class="mb-6 text-4xl md:text-5xl font-bold font-heading">
							Lorem ipsum dolor sit amet consectutar
						</h2>
						<p class="mb-8 text-lg leading-loose text-gray-500">
							Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh, pulvinar vitae
							aliquet nec, accumsan aliquet orci.
						</p>
						<div class="flex flex-wrap -mx-2">
							<div class="w-full md:w-1/2 px-2 mb-4 md:mb-0">
								<div class="flex items-center">
									<img
										class="h-12"
										src="https://shuffle.dev/uinel-assets/images/cta/avatar-woman.png"
										alt=""
									/>
									<div class="ml-4">
										<h3 class="text-2xl font-bold font-heading">
											Lorem ipsum dolor sit amet consectutar
										</h3>
										<p class="text-lg leading-loose text-gray-500">
											Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh,
											pulvinar vitae aliquet nec, accumsan aliquet orci.
										</p>
									</div>
								</div>
							</div>
							<div class="w-full md:w-1/2 px-2">
								<div class="flex items-center">
									<img
										class="h-12"
										src="https://shuffle.dev/uinel-assets/images/cta/avatar-man.png"
										alt=""
									/>
									<div class="ml-4">
										<h3 class="text-2xl font-bold font-heading">
											Lorem ipsum dolor sit amet consectutar
										</h3>
										<p class="text-lg leading-loose text-gray-500">
											Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque massa nibh,
											pulvinar vitae aliquet nec, accumsan aliquet orci.
										</p>
									</div>
								</div>
							</div>
						</div>
					</div>
				</div>
			</div>
		</div>
		<div class="navbar-menu fixed top-0 left-0 bottom-0 flex items-center justify-between z-50">
			<div class="navbar-backdrop fixed inset-0 bg-gray-800 opacity-80"></div>
			<nav class="relative flex h-32 w-full flex-col overflow-y-auto bg-white px-6 py-8">
				<div class="flex items-center mb-12">
					<a class="text-2xl font-bold" href="#">
						<img
							class="h-8"
							src="https://shuffle.dev/uinel-assets/logos/uinel-gray-light.svg"
							alt=""
						/>
					</a>
					<button class="navbar-close">
						<svg
							width="24"
							height="24"
							viewbox="0 0 24 24"
							fill="none"
							xmlns="http://www.w3.org/2000/svg"
						>
							<path
								d="M6 18L18 6M6 6L18 18"
								stroke="currentColor"
								stroke-width="2"
								stroke-linecap="round"
								stroke-linejoin="round"
							></path>
						</svg>
					</button>
				</div>
				<div class="flex flex-col justify-between">
					<div class="flex items-center">
						<a class="text-2xl font-bold" href="#">
							<img
								class="h-8"
								src="https://shuffle.dev/uinel-assets/logos/uinel-gray-light.svg"
								alt=""
							/>
						</a>
						<a
							class="group inline-flex w-full md:w-auto items-center justify-center py-3 px-5 text-sm font-medium text-gray-900 hover:text-gray-700 border border-gray-200 hover:border-gray-300 rounded-md focus:ring-2 focus:ring-gray-500 focus:ring-opacity-50"
							href="#"
							>Log In</a
						><a
							class="group inline-flex w-full md:w-auto items-center justify-center py-3 px-5 text-sm font-medium text-gray-900 hover:text-gray-700 border border-gray-200 hover:border-gray-300 rounded-md focus:ring-2 focus:ring-gray-500 focus:ring-opacity-50"
							href="#"
							>Sign Up</a
						>
					</div>
					<div class="flex flex-col justify-between">
						<a
							class="group inline-flex w-full md:w-auto items-center justify-center py-3 px-5 text-sm font-medium text-gray-900 hover:text-gray-700 border border-gray-200 hover:border-gray-300 rounded-md focus:ring-2 focus:ring-gray-500 focus:ring-opacity-50"
							href="#"
							>Log Out</a
						>
					</div>
				</div>
			</nav>
		</div>
		<div class="hidden navbar-menu fixed top-0 left-0 bottom-0 flex items-center justify-between z-50">
			<div class="navbar-backdrop fixed inset-0 bg-gray-800 opacity-80"></div>
			<nav class="relative flex h-32 w-full flex-col overflow-y-auto bg-white px-6 py-8">
				<div class="flex items-center mb-12">
					<a class="text-2xl font-bold" href="#">
						<img
							class="h-8"
							src="https://shuffle.dev/uinel-assets/logos/uinel-gray-light.svg"
							alt=""
						/>
					</a>
					<button class="navbar-close">
						<svg
							width="24"
							height="24"
							viewbox="0 0 24 24"
							fill="none"
							xmlns="http://www.w3.org/2000/svg"
						>
							<path
								d="M6 18L18 6M6 6L18 18"
								stroke="currentColor"
								stroke-width="2"
								stroke-linecap="round"
								stroke-linejoin="round"
							></path>
						</svg>
					</button>
				</div>
				<div class="flex flex-col justify-between">
					<div class="flex items-center">
						<a class="text-2xl font-bold" href="#">
							<img
								class="h-8"
								src="https://shuffle.dev/uinel-assets/logos/uinel-gray-light.svg"
								alt=""

  """))

# **Conclusion**

**Positives:**
- We can see that thanks to the finetuning of the model we are able to achieve the implementation of better designs thanks to the usage of advanced TailwindCSS components such as the floating menu and the addition of images.

- The model has successfully learned to only present the `<body>` part of the website as per the examples in the dataset, allowing us to always import our own `<head>` segment with the newest Tailwind CSS CDN address.

**Negatives:**
- We can see that the model has adopted the usage of `Lorem Ipsum` placeholders which has replaced the usage of adequate generated text.

###***A future improvement would be to gather more realistic dataset that uses real text in its website codes.***